In [3]:
%pip install --quiet pyautogen~=0.1.0

You should consider upgrading via the '/Users/jorgegarcia/VSCode/autogen-agents/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os 

llm_config={
    "request_timeout": 600,
    "seed": 44,  # change the seed for different trials
    "config_list": [
        {
            "model": "gpt-4",
            "api_key": os.environ.get("OPENAI_API_KEY"),
        }
    ],
    "temperature": 0,
}

In [3]:
from autogen import AssistantAgent, UserProxyAgent
# create an AssistantAgent instance named "assistant"
assistant = AssistantAgent(
    name="assistant",
    llm_config=llm_config,
    is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
    max_consecutive_auto_reply=10,
    code_execution_config={
        "work_dir": "work_dir",
        "use_docker": False,
    },
)

In [6]:
import pymongo

mongodb_host = "localhost"  # Your MongoDB server's address
mongodb_port = 27017  # Default MongoDB port

client = pymongo.MongoClient(mongodb_host, mongodb_port)
db = client["test"]
collection_list = db.list_collection_names()
    
task1 = f"""
Based on the user query, return a list of collections needed to answer the query.
You shold ONLY return a List of collections, nothing else.
COLLECTIONS: {collection_list}
----------------
QUERY: how many users are demo?
ANSWER: ['users']
QUERY: can you give me information about the user tech+student@ucademy.com?
ANSWER:
"""

user_proxy.initiate_chat(assistant, message=task1)

user_proxy (to assistant):


Based on the user query, return a list of collections needed to answer the query.
You shold ONLY return a List of collections, nothing else.
COLLECTIONS: ['notifications', 'messages', 'users', 'onboardings', 'groups', 'questionreviews', 'paymentplans', 'academies', 'questions', 'events', 'changelog', 'sessionlogs', 'prompts', 'verticals', 'subscriptions', 'quizzes', 'quizresults', 'activities', 'problemreports', 'courses', 'sections', 'transcriptions', 'bulletins', 'files', 'tickets', 'payments', 'certificates', 'tutoringgroups', 'tags', 'notes', 'deliverables']
----------------
QUERY: how many users are demo?
ANSWER: ['users']
QUERY: can you give me information about the user tech+student@ucademy.com?
ANSWER:


--------------------------------------------------------------------------------
assistant (to user_proxy):

['users']

--------------------------------------------------------------------------------
user_proxy (to assistant):



------------------

In [7]:
task2 = f"""
With the list of collections to query, you should run the following python script to preview the data structure.
```python
def print_schema(obj, indent):
    for key, value in obj.items():
        if not callable(value):  # No queremos imprimir funciones
            print(f'{{indent}}{{key}}: {{type(value).__name__}}')
            if isinstance(value, dict) or isinstance(value, list):  # Si la propiedad es de tipo objeto o lista, imprimir sus propiedades
                print_schema(value, f"{{indent}}\t")
                
def preview_data_structure(collection_list):
    import pymongo

    mongodb_host = "localhost"  # Your MongoDB server's address
    mongodb_port = 27017  # Default MongoDB port

    client = pymongo.MongoClient(mongodb_host, mongodb_port)
    db = client["test"]
    
    for collection in collection_list:
        sample = db[collection].find_one()
        print_schema(sample, "")
        
```
"""

user_proxy.initiate_chat(assistant, message=task2, clear_history=False)

user_proxy (to assistant):


With the list of collections to query, you should run the following python script to preview the data structure.
```python
def print_schema(obj, indent):
    for key, value in obj.items():
        if not callable(value):  # No queremos imprimir funciones
            print(f'{indent}{key}: {type(value).__name__}')
            if isinstance(value, dict) or isinstance(value, list):  # Si la propiedad es de tipo objeto o lista, imprimir sus propiedades
                print_schema(value, f"{indent}	")
                
def preview_data_structure(collection_list):
    import pymongo

    mongodb_host = "localhost"  # Your MongoDB server's address
    mongodb_port = 27017  # Default MongoDB port

    client = pymongo.MongoClient(mongodb_host, mongodb_port)
    db = client["test"]
    
    for collection in collection_list:
        sample = db[collection].find_one()
        print_schema(sample, "")
        
```


----------------------------------------------------

In [9]:
print(user_proxy.last_message())

{'content': "Great! The script has successfully printed the schema of the 'users' collection. This schema shows the structure of the data in the 'users' collection, including the field names and their data types. \n\nNow, you can use this schema to construct queries to retrieve the data you need from the 'users' collection. For example, if you want to find out how many users are demo, you can query the 'isDemo' field. If you want to get information about a specific user, you can query the 'email' field.\n\nTERMINATE", 'role': 'user'}


In [11]:
task3 = f"""
answer the user's initial query:
QUERY: can you give me information about the user tech+student@ucademy.com
"""

user_proxy.initiate_chat(assistant, message=task3, clear_history=False)

user_proxy (to assistant):


answer the user's initial query:
QUERY: can you give me information about the user tech+student@ucademy.com


--------------------------------------------------------------------------------
assistant (to user_proxy):

To answer the user's query, we can use the pymongo library to connect to the MongoDB database and retrieve the user's information. Here is the Python code to do that:

```python
import pymongo

def get_user_info(email):
    mongodb_host = "localhost"  # Your MongoDB server's address
    mongodb_port = 27017  # Default MongoDB port

    client = pymongo.MongoClient(mongodb_host, mongodb_port)
    db = client["test"]  # Replace "test" with your database name

    user = db['users'].find_one({"email": email})

    if user:
        print(user)
    else:
        print(f"No user found with email: {email}")

get_user_info("tech+student@ucademy.com")
```

Please replace "localhost" and "27017" with your MongoDB server's address and port. Also, replac

In [12]:
task4 = """Reflect on the sequence and create a recipe containing all the steps 
necessary and name for it. Suggest well-documented, generalized python function(s)
 to perform similar tasks for coding steps in future. Make sure coding steps and 
 non-coding steps are never mixed in one function. In the docstr of the function(s),
 clarify what non-coding steps are needed to use the language skill of the assistant.
"""
user_proxy.initiate_chat(assistant, message=task4, clear_history=False)

user_proxy (to assistant):

Reflect on the sequence and create a recipe containing all the steps 
necessary and name for it. Suggest well-documented, generalized python function(s)
 to perform similar tasks for coding steps in future. Make sure coding steps and 
 non-coding steps are never mixed in one function. In the docstr of the function(s),
 clarify what non-coding steps are needed to use the language skill of the assistant.


--------------------------------------------------------------------------------
assistant (to user_proxy):

Sure, here is a generalized Python function that can be used to retrieve information about a user from a MongoDB database based on their email. This function can be used in similar tasks in the future.

```python
import pymongo

def get_user_info(mongodb_host, mongodb_port, db_name, collection_name, email):
    """
    This function retrieves information about a user from a MongoDB database based on their email.

    Non-coding steps:
    - Identify t

In [8]:
import pymongo

mongodb_host = "localhost"  # Your MongoDB server's address
mongodb_port = 27017  # Default MongoDB port

client = pymongo.MongoClient(mongodb_host, mongodb_port)
db = client["test"]
collection_list = db.list_collection_names()

task='''This recipe is available for you to reuse..

<begin recipe>
**Recipe Name:** Query MongoDB

**Steps:**

1. Based on the query and the collection list extract which collections will be involved.
2. For each collection, analyze its data structure by calling "preview_data_structure".
3. Do NOT try try to make any assumptions about the data structure. Just print it out.
3. Once you have analyzed the data structure, run a python script to perform querys.
    You should consider that mongo querys are CASE SENSITIVE.
4. Generate a written response with the data obtained.

Here are the well-documented, generalized Python functions to perform the coding steps in the future:

def print_schema(obj, indent):
    if isinstance(obj, dict):
        for key, value in obj.items():
            if not callable(value):  # We don't want to print functions
                print(f'{indent}{key}: {type(value).__name__}')
                if isinstance(value, dict):
                    print_schema(value, f"{indent}	")
                elif isinstance(value, list) and value:
                    print(f'{indent}{key}: List of {type(value[0]).__name__}')
    elif isinstance(obj, list) and obj:
        print(f'{indent}List of {type(obj[0]).__name__}')

def preview_data_structure(collection: str):
    import pymongo

    mongodb_host = "localhost"  # Your MongoDB server's address
    mongodb_port = 27017  # Default MongoDB port

    client = pymongo.MongoClient(mongodb_host, mongodb_port)
    db = client["test"]
    
    
    sample = db[collection].find_one()
    print_schema(sample, "")
    
**Usage:**

1. Use the `preview_data_structure` function to print a collection schema.
2: COLLECTIONS: {collection_list}
</end recipe>


Here is a new task:
How many quizzes have been done between 1 Sep 2023 and 10 Sep 2023?
'''
user_proxy.initiate_chat(assistant, message=task, clear_history=True)
print(user_proxy.last_message())

user_proxy (to assistant):

This recipe is available for you to reuse..

<begin recipe>
**Recipe Name:** Query MongoDB

**Steps:**

1. Based on the query and the collection list extract which collections will be involved.
2. For each collection, analyze its data structure by calling "preview_data_structure".
3. Do NOT try try to make any assumptions about the data structure. Just print it out.
3. Once you have analyzed the data structure, run a python script to perform querys.
    You should consider that mongo querys are CASE SENSITIVE.
4. Generate a written response with the data obtained.

Here are the well-documented, generalized Python functions to perform the coding steps in the future:

def print_schema(obj, indent):
    if isinstance(obj, dict):
        for key, value in obj.items():
            if not callable(value):  # We don't want to print functions
                print(f'{indent}{key}: {type(value).__name__}')
                if isinstance(value, dict):
                 

In [7]:
print(user_proxy.last_message())

{'content': 'Great! The script has successfully counted the number of quizzes. \n\nAccording to the output, there have been 1926 quizzes done.\n\nPlease let me know if you need help with anything else.\n\nTERMINATE', 'role': 'user'}
